In [3]:
#importing packages
from collections import Counter, defaultdict
import math
import numpy as np
import os
import pandas as pd
import re
from scipy.sparse import csr_matrix
#import urllib.request
#import zipfile

# Use nltk for valid words
import nltk
# Need to make hash 'dictionaries' from nltk for fast processing
import collections as co
import csv


In [4]:
# Read the input
d = pd.read_csv("//home//swapnil//Documents//CS522//consumer_complaints.csv") # the consumer dataset is now a Pandas DataFrame
# Only interested in data with consumer complaints
d = d[d['Consumer complaint narrative'].notnull()]

/home/swapnil/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print(d['Consumer complaint narrative'])

56777     Received Capital One charge card offer XXXX. A...
56834     I do n't know how they got my cell number. I t...
56884     I 'm a longtime member of Charter One Bank/RBS...
56894     After looking at my credit report, I saw a col...
56898     I received a call from a XXXX XXXX from XXXX @...
56910     Was not contacted 4 years later about some pri...
56911     Collection Consultants is reporting a collecti...
56914     I had my purse stolen in 2007. They never foun...
56917     I attempted to apply for a Discover Card Onlin...
56918     Continued attempts by XXXX XXXX XXXX to collec...
56920     This is a continuation of a previous issue wit...
56928     Going through a divorce, my ex and I were unab...
56930     I am assisted with my mortgage through an agen...
56939     i submitted a payment on XXXX XXXX 2015 for my...
56940     I recieved a notice from Midland Credit Manage...
56956     XXXX Card services was bought out by Capital O...
56965     I was reported late by Discove

In [6]:
print(d[1:5])

      Date received           Product                            Sub-product  \
56834     3/23/2015   Debt collection  Other (i.e. phone, health club, etc.)   
56884     3/23/2015       Credit card                                    NaN   
56894     3/23/2015  Credit reporting                                    NaN   
56898     3/23/2015   Debt collection                            Payday loan   

                                        Issue  \
56834     Improper contact or sharing of info   
56884                                 Rewards   
56894  Incorrect information on credit report   
56898     Improper contact or sharing of info   

                                   Sub-issue  \
56834      Contacted me after I asked not to   
56884                                    NaN   
56894                Information is not mine   
56898  Talked to a third party about my debt   

                            Consumer complaint narrative  \
56834  I do n't know how they got my cell number. I 

In [7]:
def tokenize_string(my_string):
    """ DONE. You should use this in your tokenize function.
    """
    #return re.findall('[\w\-]+', my_string.lower())
    #\W -> Matches any non-alphanumeric character; 
    #this is equivalent to the class [^a-zA-Z0-9_]. 
      
    # preprocessing ->
    # remove numbers    
    word_list = re.sub('[^a-zA-Z]+', ' ', my_string.lower()).split()
    
   
    # remove stop words
    stopWords = co.Counter( nltk.corpus.stopwords.words() )

    #print(stopWords)
    
    clean_word_list = [x for x in word_list if x not in stopWords]
    
    #print(clean_word_list)
    
    
    # remove small and large words
   
    word_list = [x for x in clean_word_list if len(x) > 4 and len(x) < 15 ]
    
    
    # remove xxxx
    
    clean_word_list = [x for x in word_list if "xxxx" not in x ]
   
    # remove too low and too high frequency words -> we can't calculate here

    
    
    print(clean_word_list)
    
    return(clean_word_list)

In [8]:
def tokenize(data):
    """
    Append a new column to the movies DataFrame with header 'tokens'.
    This will contain a list of strings, one per token, extracted
    from the 'genre' field of each movie. Use the tokenize_string method above.
    Note: you may modify the movies parameter directly; no need to make
    a new copy.
    Params:
      movies...The movies DataFrame
    Returns:
      The movies DataFrame, augmented to include a new column called 'tokens'.
    >>> movies = pd.DataFrame([[123, 'Horror|Romance'], [456, 'Sci-Fi']], columns=['movieId', 'genres'])
    >>> movies = tokenize(movies)
    >>> movies['tokens'].tolist()
    [['horror', 'romance'], ['sci-fi']]
    """
    ###TODO
    
    # step 1 -> do tokenize_string for each row in movies['genres']

    all_words = []
    for row in data['Consumer complaint narrative']:
        #genre_list = re.sub(r'[||)|(]', r' ',row.lower()).split()
        word_list = tokenize_string(row)
        #print(word_list)
        #print(len(genre_list))
        all_words.append(word_list)
    
    # step 2 -> add column tokens in movies
    array = np.array(all_words)
    
    #print(array[:5])
    #print('#list = ',len(array))
    
    new_data = data.assign(tokens = array)
    
    #print(new_movies.head(5))
    
    print(new_data[1:5])
    return(new_data)  
    pass

In [9]:
def featurize(data):
    """
    Append a new column to the movies DataFrame with header 'features'.
    Each row will contain a csr_matrix of shape (1, num_features). Each
    entry in this matrix will contain the tf-idf value of the term, as
    defined in class:
    tfidf(i, d) := tf(i, d) / max_k tf(k, d) * log10(N/df(i))
    where:
    i is a term
    d is a document (movie)
    tf(i, d) is the frequency of term i in document d
    max_k tf(k, d) is the maximum frequency of any term in document d
    N is the number of documents (movies)
    df(i) is the number of unique documents containing term i
    Params:
      movies...The movies DataFrame
    Returns:
      A tuple containing:
      - The movies DataFrame, which has been modified to include a column named 'features'.
      - The vocab, a dict from term to int. Make sure the vocab is sorted alphabetically as in a2 (e.g., {'aardvark': 0, 'boy': 1, ...})
   
    """
    ###TODO
    #print(movies[:5]) 
    
    #step 1 -> build a vocab and df(term)
    vocab = {}
    vocab_list = []
    df = {}
    
    for row1 in data['tokens']:
        row = list(set(row1))
        for term in row:
            if term not in vocab.keys():
                vocab.setdefault(term,-1)
             
            if term not in df.keys(): 
                df.setdefault(term,1)
            else :
                df[term] += 1
             
             
    #print('vocab = ', vocab)
    
    vocab_list = sorted(vocab.keys(), key = lambda x:x)
    #print('vocab_list = ', vocab_list)
    
    for i,term in enumerate(vocab_list):
         vocab[term] = i
            
    #        
         
    #print('Sorted vocab = ', sorted(vocab.items()))
    #print('df = ',sorted(df.items(), key=lambda x:x[0]))
    
    # step 2 -> Build a csr_matrix for each row of movies['tokens']
    N = len(data)
    #print('N = ',N)
    
    #[comedy, comedy, comedy, horror]  -> max_k tf(k, d) = 3 
    #[action, comedy,thriller] -> tf(action, d) =1
    # df(i) ->
    #num_features is the total number of unique features across all documents.
    
    csr_array =[]
    
    for row1 in data['tokens']:
        csr_row = []
        csr_col = []
        csr_data = []
        max_k = 0
       
        #print(row1)
        
        if row1 != []:
            max_k = Counter(row1).most_common()[:1][0][1]
            row = list(set(row1))

            #print(max_k)

            #print('removed duplicates =',row)
            for term in row:       
                csr_row.append(0)
                csr_col.append(vocab[term])
                #tfidf(i, d) := tf(i, d) / max_k tf(k, d) * log10(N/df(i))
                tf = Counter(row1)[term]
                #max_k = max_k.most_common()[:1][0][1]
         
                #print('term = %s ---> tf = %d ---> max_k = %d'%(term,tf,max_k))
                tfidf = (tf / max_k) * math.log10(N/df[term])
                csr_data.append(tfidf)
           

         
        X = csr_matrix((csr_data, (csr_row, csr_col)), shape=(1, len(vocab)), dtype=np.float64)
       
        #print('X ->\n',X.toarray())
        csr_array.append(X)
    

    # step 3 -> add column features to movies 
    #print('size of csr_array = ',len(csr_array)) 
    #print('CSR = ',csr_array[:2])  
    new_data = data.assign(features = csr_array)
    #print(new_movies.head(2))
     
    return(new_data,vocab)   
    
    pass

In [10]:
d1 = d[1:4000]

In [11]:
#print(d1)

In [12]:
data = tokenize(d1)

['number', 'would', 'onlybwith', 'original', 'company', 'harassing', 'called', 'sending', 'contact', 'messsage', 'would', 'report', 'contacted', 'people', 'whose', 'concern', 'business']
['longtime', 'member', 'charter', 'citizens', 'mother', 'became', 'elderly', 'assisted', 'opening', 'account', 'charter', 'branch', 'office', 'located', 'michigan', 'placed', 'account', 'account', 'owner', 'mother', 'became', 'necessary', 'obtain', 'durable', 'power', 'attorney', 'handle', 'financial', 'medical', 'needs', 'years', 'mother', 'present', 'presented', 'original', 'stamped', 'durable', 'power', 'attorney', 'charter', 'branch', 'office', 'located', 'michigan', 'record', 'presented', 'understanding', 'would', 'placed', 'mother', 'account', 'furthermore', 'needed', 'transact', 'business', 'would', 'welcomed', 'though', 'contacted', 'charter', 'master', 'someone', 'fraudulent', 'charges', 'mother', 'credit', 'account', 'caller', 'stated', 'would', 'immediately', 'close', 'mother', 'master', 'cr

In [13]:
print(data['tokens'][1:5])

56884    [longtime, member, charter, citizens, mother, ...
56894    [looking, credit, report, collection, account,...
56898    [received, stating, wanted, however, would, as...
56910    [contacted, years, later, private, supposedly,...
Name: tokens, dtype: object


In [14]:
data, vocab = featurize(data)

In [15]:
print('vocab:')
print(sorted(vocab.items())[:500])

vocab:
[('aacceptance', 0), ('aacount', 1), ('aargon', 2), ('aavings', 3), ('abanding', 4), ('abandoned', 5), ('abandoning', 6), ('abandonment', 7), ('abide', 8), ('abided', 9), ('abiding', 10), ('abilities', 11), ('ability', 12), ('abnormal', 13), ('abouts', 14), ('aboutthis', 15), ('abrasive', 16), ('abroad', 17), ('abrogated', 18), ('abrupt', 19), ('abruptly', 20), ('absence', 21), ('absences', 22), ('absent', 23), ('absolute', 24), ('absolutely', 25), ('absolve', 26), ('absorve', 27), ('abstract', 28), ('absurd', 29), ('absurdity', 30), ('absurdly', 31), ('abundantly', 32), ('abuse', 33), ('abused', 34), ('abuses', 35), ('abusing', 36), ('abusive', 37), ('abusively', 38), ('abysmal', 39), ('acccording', 40), ('acccount', 41), ('accelerate', 42), ('accelerated', 43), ('acceleration', 44), ('accent', 45), ('accept', 46), ('acceptable', 47), ('acceptace', 48), ('acceptal', 49), ('acceptance', 50), ('accepted', 51), ('accepting', 52), ('accepts', 53), ('acces', 54), ('access', 55), ('a

In [16]:
print(len(vocab))

10773


In [17]:
print(data['features'][1:5])

56884      (0, 39)\t0.300162617011\n  (0, 83)\t0.300825...
56894      (0, 83)\t0.100275226511\n  (0, 97)\t0.281570...
56898      (0, 724)\t0.363734793217\n  (0, 1146)\t0.881...
56910      (0, 307)\t0.498862675525\n  (0, 535)\t1.8615...
Name: features, dtype: object


In [18]:
def train_test_split(data):
    """DONE.
    Returns a random split of the ratings matrix into a training and testing set.
    """
    test = set(range(len(data))[::10])
    train = sorted(set(range(len(data))) - test)
    test = sorted(test)
    return data.iloc[train], data.iloc[test]

In [19]:
data_train, data_test = train_test_split(data)
print('%d training ratings; %d testing ratings' % (len(data_train), len(data_test)))

3599 training ratings; 400 testing ratings


In [20]:
def cosine_sim(a, b):
    """
    Compute the cosine similarity between two 1-d csr_matrices.
    Each matrix represents the tf-idf feature vector of a movie.
    Params:
      a...A csr_matrix with shape (1, number_features)
      b...A csr_matrix with shape (1, number_features)
    Returns:
      The cosine similarity, defined as: dot(a, b) / ||a|| * ||b||
      where ||a|| indicates the Euclidean norm (aka L2 norm) of vector a.
      
    >>> a = csr_matrix([[ 0.0,  0.0,  0.0,  0.0, 0.43974934,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0, 0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  1.3955668,  0.0]] )
    >>> b = csr_matrix([[ 0.0,  0.0,  0.0,  0.0,  0.43974934,  0.0,  0.0,  0.32024863,  0.0, 0.0,  0.0,  0.0, 0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0]] )
    >>> sim = cosine_sim(a, b)
    >>> print(sim)
    0.242942001121
    >>> a =  csr_matrix([[ 0.0,  0.0,        0.0,        0.0,        0.43974934,  0.0,  0.0,  0.0,         0.0, 0.0,  0.0,  0.0,  0.0,        0.0,  0.0,        0.0,  0.0,  0.0,  0.0,  0.0,  1.3955668,  0.0]] )
    >>> b =  csr_matrix([[ 0.0,  0.9121797,  1.3099253,  1.1945643,  0.0,         0.0,  0.0,  0.32024863,  0.0,  0.0,  0.0,  0.0,  1.7755414,  0.0,  1.3647367,  0.0,  0.0,  0.0,  0.0,  0.0,  0.0,        0.0]] )
    >>> sim = cosine_sim(a, b)
    >>> print(sim)
    0.0
    """
    ###TODO
    #print(type(a))
    #print(type(b))
    #print('a = ',a)
    #print('b = ',b)
    
    #step 1 -> calculate ||a||
    x = (a.data * a.data)
    X = math.sqrt(x.sum())    
    #print('X = ',X)

    #step 2 -> calculate ||b||
    y = (b.data * b.data)
    Y = math.sqrt(y.sum())    
    #print('Y = ',Y)

    # step 3 -> calculate dot(a, b)
    #print('a = ',a.todense())
    #print('b = ',b.todense())
    #print('a = ',a.shape)
    #print('b = ',b.shape)
    
    
    dotProduct = (a).dot(b.transpose())
    #print('dotProduct = ',dotProduct)

    Sum_dot = dotProduct.sum()
    #print('Sum of dotProduct = ',Sum_dot)
    
    # step 4 -> calculate cosine similarity
    if (X != 0) | (Y !=0) :
        Cos_Sim = Sum_dot / ( X * Y )
    else :
        Cos_Sim = 0
    
    #print('Cos_Sim = ',Cos_Sim)
    return(Cos_Sim)
    
    pass

In [21]:
print(data[1:5])

      Date received           Product               Sub-product  \
56884     3/23/2015       Credit card                       NaN   
56894     3/23/2015  Credit reporting                       NaN   
56898     3/23/2015   Debt collection               Payday loan   
56910     3/23/2015      Student loan  Non-federal student loan   

                                        Issue  \
56884                                 Rewards   
56894  Incorrect information on credit report   
56898     Improper contact or sharing of info   
56910      Dealing with my lender or servicer   

                                   Sub-issue  \
56884                                    NaN   
56894                Information is not mine   
56898  Talked to a third party about my debt   
56910  Having problems with customer service   

                            Consumer complaint narrative  \
56884  I 'm a longtime member of Charter One Bank/RBS...   
56894  After looking at my credit report, I saw a col... 

In [41]:
def make_predictions(data, data_train, data_test):
    """
    Using the ratings in ratings_train, predict the ratings for each
    row in ratings_test.
    To predict the rating of user u for movie i: Compute the weighted average
    rating for every other movie that u has rated.  Restrict this weighted
    average to movies that have a positive cosine similarity with movie
    i. The weight for movie m corresponds to the cosine similarity between m
    and i.
   
    If there are no other movies with positive cosine similarity to use in the
    prediction, use the mean rating of the target user in ratings_train as the
    prediction.
    Params:
      movies..........The movies DataFrame.
      ratings_train...The subset of ratings used for making predictions. These are the "historical" data.
      ratings_test....The subset of ratings that need to predicted. These are the "future" data.
    Returns:
      A numpy array containing one predicted rating for each element of ratings_test.
    """
    
    ###TODO
    #print('movies = \n',movies.head(2))
    #print('ratings_train = \n',ratings_train.head(2))
    #print('ratings_test = \n',ratings_test.head(2))
    
    # steps ->  traverse each row of ratings_test 
    # find user's movie To predict and his movie list that he rated 
    # and then find cosine sim
    class1 = 0
    class2 = 0
    class3 = 0   
    class4 = 0
    class5 = 0
    class6 = 0   
    class7 = 0   
    class8 = 0
   
    result = [] 
    wrong_pred = 0
    
    for index, row in data_test.iterrows():
        Complain_ID = row['Complaint ID']
        product = row['Product']
        issue = row['Issue'] 
       
        #user_movieTopredict = movies[(movies.movieId == test_movie)]
        #a =  user_movieTopredict['features'].values[0]
        
        responce_Topredict = data[(data['Complaint ID'] == Complain_ID )]
        a =  responce_Topredict['features'].values[0]

        trains_data = data_train[ (data_train['Complaint ID'] != Complain_ID) & ((data_train.Product == product) | (data_train.Issue == issue))]
        
        #print("Test Data")
        #print(Complain_ID)
        #print(product)
        #print(issue)          
        #print(trains_data['Complaint ID'])
    

        update_cos_sim = 0
        higest_cos_match = 0
        
        for index1, row1 in trains_data.iterrows():          
                data_Hist = data[(data['Complaint ID'] == row1['Complaint ID'])]
                #print("History Data")
                #print(data_Hist['Product'])
                #print(data_Hist['Complaint ID'])
                b =  data_Hist['features'].values[0]


                cos_sim = cosine_sim(a,b)
                #print(cos_sim) 
              
                 
                if (cos_sim > 0) & (update_cos_sim < cos_sim ):
                        update_cos_sim = cos_sim   
                        #print(update_cos_sim)   
                        #print(row1['Product'])
                        #print(row1['Complaint ID'])
                        higest_cos_match = row1['Complaint ID']
                        
        #print("Cos_Sim",update_cos_sim)            
        #print('Complaint_ID=%d test_product=%s \nActual_responce=%s \nPred_responce=%s'%(Complain_ID,product,row['Company response to consumer'],row1['Company response to consumer']))  
        result.append((higest_cos_match,update_cos_sim))
        
        
        for index2, row2 in trains_data.iterrows():
            responce = ''
            if higest_cos_match == row2['Complaint ID'] :
                responce =  row2['Company response to consumer']
                #print(higest_cos_match)
                #print('highest match customer id = ',higest_cos_match)
                    
                if row['Company response to consumer'] != responce :
                    wrong_pred = wrong_pred + 1
                    #print("Actual Responce -> ",(row['Company response to consumer'])) 
                    #print("Responce Predicted -> ",(responce)) 
                
                if row['Company response to consumer'] == responce :
                    #print(row['Company response to consumer'])
                    if   'Closed with explanation'.lower()   in row['Company response to consumer'].lower() :
                        class1 = class1 + 1;
                    if  'Closed with non-monetary relief'.lower()   in row['Company response to consumer'].lower() :
                        class2 = class2 + 1;
                    if  'Closed with monetary relief'.lower()   in row['Company response to consumer'].lower() :
                        class3 = class3 + 1;
                    if  'Untimely response'.lower()  in row['Company response to consumer'].lower() :
                        class4 = class4 + 1;
                    if  'Closed without relief'.lower()   in row['Company response to consumer'].lower() :
                        class5 = class5 + 1; 
                    if  'Closed with relief'.lower()   in row['Company response to consumer'].lower() :
                        class6 = class6 + 1;
                    if  'In progress'.lower()   in row['Company response to consumer'].lower() :
                        class7 = class7 + 1;
                    if  'Closed'.lower() == row['Company response to consumer'].lower() :
                        class8 = class8 + 1;

    
    print("*** Test Data Classification ***")                
    print('Closed with explanation = ',class1)
    print('Closed with non-monetary relief = ',class2)
    print('Closed with monetary relief = ',class3)
    print('Untimely response = ',class4)
    print('Closed without relief = ',class5)
    print('Closed with relief = ',class6)
    print('In progress = ',class7)  
    print('Closed = ',class8)
    
    class12 = 0
    class22 = 0
    class32 = 0   
    class42 = 0
    class52 = 0
    class62 = 0   
    class72 = 0   
    class82 = 0
    
    
    print("*** Test Data Classification Accuracy ***")   
    for index3, row3 in data_test.iterrows():
        if   'Closed with explanation'.lower()   in row3['Company response to consumer'].lower() :
             class12 = class12 + 1;
        if  'Closed with non-monetary relief'.lower()   in row3['Company response to consumer'].lower() :
             class22 = class22 + 1;
        if  'Closed with monetary relief'.lower()   in row3['Company response to consumer'].lower() :
             class32 = class32 + 1;
        if  'Untimely response'.lower()  in row3['Company response to consumer'].lower() :
             class42 = class42 + 1;
        if  'Closed without relief'.lower()   in row3['Company response to consumer'].lower() :
             class52 = class52 + 1; 
        if  'Closed with relief'.lower()   in row3['Company response to consumer'].lower() :
             class62 = class62 + 1;
        if  'In progress'.lower()   in row3['Company response to consumer'].lower() :
             class72 = class72 + 1;
        if  'Closed'.lower() == row3['Company response to consumer'].lower() :
             class82 = class82 + 1;
     
    print('Wrong Predictions -Closed with explanation = ',class12-class1)
   
    if class12 != 0 :
        print('Error % Closed with explanation = ',((class12-class1)/class12)*100 )
    else :
        print('Error % Closed with explanation = 0')

    print('Wrong Predictions -Closed with non-monetary relief = ',class22-class2)
        
    if class22 != 0 :
        print('Error % Closed with non-monetary relief = ',((class22-class2)/class22)*100 )
    else :
        print('Error % Closed with monetary relief = 0')

    print('Wrong Predictions -Closed with monetary relief = ',class32-class3)
        
    if class32 != 0 :
        print('Error % Closed with monetary relief = ',((class32-class3)/class32)*100 )
    else :
        print('Error % Closed with monetary relief = 0')

    print('Wrong Predictions -Untimely response = ',class42-class4)
       
    if class42 != 0 :
        print('Error % Untimely response = ',((class42-class4)/class42)*100 )
    else :
        print('Error % Closed with monetary relief = 0')

    print('Wrong Predictions- Closed without relief = ',class52-class5)
      
    if class52 != 0 :
        print('Error % Closed without relief = ',((class52-class5)/class52)*100 )
    else :
        print('Error % Closed with monetary relief = 0')

    print('Wrong Predictions -Closed with relief = ',class62-class6)
        
    if class62 != 0 :    
        print('Error % Closed with relief = ',((class62-class6)/class62)*100 )
    else :
        print('Error % Closed with monetary relief = 0')

    print('Wrong Predictions -In progress = ',class72-class7)  
       
    if class72 != 0 :        
        print('Error % In progress = ',((class72-class7) /class72)*100 ) 
    else :
        print('Error % Closed with monetary relief = 0')

    print('Wrong Predictions -Closed = ',class82-class8)
       
    if class82 != 0 :
        print('Error % Closed = ',((class82-class8)/class82)*100 )    
    else :
        print('Error % Closed with monetary relief = 0')
    
    
    return(result,wrong_pred)    


    pass



In [42]:
predictions,wrong_pred = make_predictions(data, data_train, data_test)

*** Test Data Classification ***
Closed with explanation =  239
Closed with non-monetary relief =  15
Closed with monetary relief =  7
Untimely response =  0
Closed without relief =  0
Closed with relief =  0
In progress =  0
Closed =  3
*** Test Data Classification Accuracy ***
Wrong Predictions -Closed with explanation =  70
Error % Closed with explanation =  22.653721682847898
Wrong Predictions -Closed with non-monetary relief =  39
Error % Closed with non-monetary relief =  72.22222222222221
Wrong Predictions -Closed with monetary relief =  12
Error % Closed with monetary relief =  63.1578947368421
Wrong Predictions -Untimely response =  3
Error % Untimely response =  100.0
Wrong Predictions- Closed without relief =  0
Error % Closed with monetary relief = 0
Wrong Predictions -Closed with relief =  0
Error % Closed with monetary relief = 0
Wrong Predictions -In progress =  0
Error % Closed with monetary relief = 0
Wrong Predictions -Closed =  12
Error % Closed =  80.0


In [39]:
def mean_error(wrong_pred, data_test):
    """DONE.
    Return the mean absolute error of the predictions.
    """
    error = 0.0
    
    error = (wrong_pred / len(data_test) ) * 100
    
    return error


In [40]:
print('error=%f' % mean_error(wrong_pred, data_test))
print(predictions[:10])

error=34.000000
[(1323810.0, 0.13922357039950148), (1374428.0, 0.18154337727180789), (1377269.0, 0.22407554075333289), (1358311.0, 0.39704578390511058), (1311058.0, 0.20148570088726453), (1299258.0, 0.11928601606225367), (1301649.0, 0.96045036183606469), (1383347.0, 0.21179084846448998), (1311000.0, 0.27609915370256621), (1336098.0, 0.2490472082175543)]
